# Solve RCPSP by linear programming: MILP

We admit that you followed the [first notebook](RCPSP%20%231%20Introduction.ipynb) that introduced you all the basics for RCPSP Problems, on which we will apply here linear programming.

Linear programming is a paradigm to model optimisation problem where the objective function and constraints are linear in terms of the variables : 

$ y = max_{x}(c^t.x) $
such that $A.x \leq b $

$x$ can be either floating or integer values, which explains the Mixed Integer Linear Programming denomination.

## Prerequisites


Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with discrete-optimization and minizinc.
- If running on colab, the next cell does it for you.
- If running on binder, the environment should be ready.


In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import importlib
    import os
    import sys  # noqa: avoid having this import removed by pycln

    !{sys.executable} -m pip install -U pip

    # uninstall google protobuf conflicting with ray and sb3
    ! pip uninstall -y protobuf

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install git+https://github.com/airbus/discrete-optimization@master#egg=discrete-optimization

    # be sure to load the proper cffi (downgraded compared to the one initially on colab)
    import cffi

    importlib.reload(cffi)

    # install and configure minizinc
    !curl -o minizinc.AppImage -L https://github.com/MiniZinc/MiniZincIDE/releases/download/2.6.3/MiniZincIDE-2.6.3-x86_64.AppImage
    !chmod +x minizinc.AppImage
    !./minizinc.AppImage --appimage-extract
    os.environ["PATH"] = f"{os.getcwd()}/squashfs-root/usr/bin/:{os.environ['PATH']}"
    os.environ["LD_LIBRARY_PATH"] = (
        f"{os.getcwd()}/squashfs-root/usr/lib/:{os.environ['LD_LIBRARY_PATH']}"
    )

### Imports

In [ ]:
import logging

import matplotlib.pyplot as plt
import nest_asyncio

from discrete_optimization.datasets import fetch_data_from_psplib
from discrete_optimization.rcpsp.rcpsp_parser import get_data_available, parse_file
from discrete_optimization.rcpsp.rcpsp_utils import plot_ressource_view
from discrete_optimization.rcpsp.solver.rcpsp_lp_solver import LP_RCPSP, ParametersMilp

# patch asyncio so that applications using async functions can run in jupyter
nest_asyncio.apply()

# set logging level
logging.basicConfig(level=logging.INFO)

### Download datasets

If not yet available, we import the datasets from [psplib](https://www.om-db.wi.tum.de/psplib/data.html).

In [ ]:
needed_datasets = ["j301_1.sm"]
download_needed = False
try:
    files_available_paths = get_data_available()
    for dataset in needed_datasets:
        if len([f for f in files_available_paths if dataset in f]) == 0:
            download_needed = True
            break
except:
    download_needed = True

if download_needed:
    fetch_data_from_psplib()

In [ ]:
# Parse some rcpsp file
filepath = [f for f in get_data_available() if "j601_1.sm" in f][0]
rcpsp_problem = parse_file(filepath)

## LP Solver 

We have a few MILP model/solver available for single mode RCPSP, ```LP_RCPSP``` and ```LP_MRCPSP``` are using [mip](https://www.python-mip.com) python library and are adapted to single and multimode RCPSP.
```LP_MRCPSP_GUROBI``` is using gurobi solver and can be used if you have the required licence. In this notebook we'll use open source solver ```CBC``` via the `mip` based solver.

The LP modelling of the problem is based on time-indexed binary variable : $x[task, time]$ indicates if the given task is started at the given time.
Constraints of *RCPSP* can be then written with a bit of effort. The LP models are written in ```model.init_model``` function of the solvers.

If we note $c(task, r)$ the consumption of ressource $r$ of a task and $cap(r)$ the capacity of the resource $r$ : 

- Precedence constraints 
$\forall t_1, \forall t_2 \in \text{successors}(t_1) \sum_{t} t.x[t_2,t] \geq \sum_{t} t.x[t_1,t]$
- Cumulative constraints
$\forall r \in \text{ressources}, \forall t\in[0,horizon] \sum_{task}c(task,r)*\sum_{t'\in[t-dur(task)+1, t]}x[task,t'] \leq cap(r)$
Here the second sum represents the fact that the task is running at time $t$ and thus is contributing by $c(task,r)$ to the resource co


In [ ]:
solver = LP_RCPSP(problem=rcpsp_problem)
parameters_milp = ParametersMilp.default()
solver.init_model(greedy_start=True)
results = solver.solve(parameters_milp=parameters_milp)

### Plot the quality of solutions found by LP solver

In [ ]:
fig, ax = plt.subplots(1)
ax.plot([x[1] for x in results.list_solution_fits[::-1]], marker="o")
ax.set_ylabel("- makespan")
ax.set_xlabel("# solution found")
plt.show()

In [ ]:
# Retrieve the best_solution, fit found by the solver.
best_solution, fit = results.get_best_solution_fit()
# Print fitness
print(fit)
# Check if the solution satisfies the constraints
print("Satisfaction :", rcpsp_problem.satisfy(best_solution))
# Evaluation :
print("Evaluation :", rcpsp_problem.evaluate(best_solution))

### Plot the solution

In [ ]:
plot_ressource_view(rcpsp_model=rcpsp_problem, rcpsp_sol=best_solution)

Linear programming is a suitable solving approach for scheduling approaches, though it can lead to some bottlenecks when number of variables increase : here the time indexed approach would lose efficiency when the horizon is one order of magnitude higher !
Other linear formulation could be implemented that has better scaling property.